In [1]:
#-*- coding:utf-8 -*-

import requests, os
import bs4
from bs4 import BeautifulSoup

import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from urllib import parse
import re
import lxml
import json
import sys
import os

In [2]:
class Instagram_crawler:
    def crawling_data(self, tag, no_of_scroll, start_page):
        
        self.tag = tag
        self.tag_url = "www.instagram.com/explore/tags/" + self.tag
        self.tag_url = "http://" + parse.quote(self.tag_url)
        print('\n', self.tag_url, '\n')
        
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        option.add_argument('disable-gpu')

        self.browser = webdriver.Chrome('chromedriver', options=option)

        # browser = webdriver.Chrome()
        self.browser.get(self.tag_url)
        time.sleep(3)
        
        print('start to get shortcode')
        
        for i in range(1, start_page):
            self.scroll_page()
        
        i = 0
        total_saved_data = 0
        shortcodes = []
        while i < no_of_scroll:
            # get html code
            soup = self.scroll_page()
            # get the list of shortcode
            shortcodes.extend(self.get_shortcode_list(soup))
            print()
            i+=1
        
        print('\nstart to get data')
        saved_data = self.save_data(shortcodes)
        print('  saved data :', saved_data)
        
    # scroll url page num times and get html data
    def scroll_page(self):
        body = self.browser.find_element_by_tag_name("body")   
        no_of_pagedowns = 8;
        
        while no_of_pagedowns:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
            no_of_pagedowns-=1

        response = self.browser.page_source
        soup = BeautifulSoup(response, 'lxml')
        print(" read the HTML file")

        return soup
    
    def create_folder(self, directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
    
    # return the list of shortcode from html code
    def get_shortcode_list(self, soup):
        article = soup.find("article")
        href = article.find_all('a')

        shortcodes = []

        for h in href:
            shortcodes.append(h['href'])

        print(" get ", len(shortcodes) - 9, " shortcodes")

        return shortcodes[9:]


    # return the text from html code
    def get_text(self, page):
        script = page.find_all("script")[3].string
        script_split = re.split(':|,|\"|\'|\{|\}|\[|\]',script)  
        text_index = script_split.index('text') + 3
        text = script_split[text_index]  

        text = text.replace('\\n','')

        text = text.encode('utf-8', 'surrogatepass')
        text = text.decode('unicode_escape')
        
        words = text.replace("#"," #")
        words = re.split(' ',words)          
        hashtag = []        
        for w in words:
            if '#' in w:
                hashtag.append(w)
        
        return text, hashtag

    # return the user_id and number of likes from html code
    def get_id_likes(self, description):
        data = re.split('on Instagram:', description)

        data = re.split(",|\(|\)|@|Likes|Comments|-| ",data[0])
        while '' in data : data.remove('')

        likes = data[0]
        user_id = data[-1]

        return user_id, likes

    # return the text, user_id, number of likes from html code
    def get_labels(self, page):
        for tag in page.select('meta[property*=image]'):
            image_url = tag['content']

        for tag in page.select('meta[property*=og:description]'):
            description = tag['content']

        text, hashtag = self.get_text(page)
        user_id, likes = self.get_id_likes(description)

        return text, hashtag, user_id, likes

    # save image in page
    def save_image(self, page, num):
        for tag in page.select('meta[property*=image]'):
            image_url = tag['content']

        pic_response = requests.get(image_url).content
        img_name = "./dataset/img/" + self.tag + "/"+ str(num) + ".jpg"

        with open(img_name, 'wb') as f:
            f.write(pic_response)
        print(" file saved: ", img_name)    

    # get html file of "https://www.instagram.com" + shortcode page, and get data (likes, user_id, text, hashtag)
    def save_data(self, shortcodes):
        i = 1
        saved_data = 0
        file_name = "./dataset/label_" + self.tag +  ".json"
        self.create_folder("./dataset/img/" + self.tag)
        
        for shortcode in shortcodes:
            print("[ ", i, ' ]')
            i = i + 1  
            label_data = []

            try:
                # get html code
                page_url = "https://www.instagram.com" + shortcode
                print(' ', page_url)
                response = requests.get(page_url).content
                page = BeautifulSoup(response, 'lxml')        

                # get label data
                text, hashtag, user_id, likes = self.get_labels(page)

                # save label data 
                output = {"page_url":page_url, "img_name":saved_data, "label":self.tag, "likes":likes, "user_id":user_id, "text":text, "hashtag":hashtag}
                label_data.append(output)

                if not os.path.isfile(file_name):
                    with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                        data = []
                        data.append(label_data)
                        new_feed = json.dumps(data, ensure_ascii=False, indent=2)
                        f.write(new_feed)
                else:
                    with open(file_name, encoding='UTF-16') as feedsjson:
                        old_feeds = []
                        old_feeds.extend(json.load(feedsjson))
                    old_feeds.append(label_data)
                    with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                        new_feed = json.dumps(old_feeds, ensure_ascii=False, indent=2)
                        f.write(new_feed) 

                # save image files
                self.save_image(page, saved_data)

                print(" Okay\n")
                saved_data += 1

            except:
                print(" Unexpected error:", sys.exc_info()[0], sys.exc_info()[1])
                print(" Line ", sys.exc_info()[-1].tb_lineno)
                print(" Not okay\n")

        return saved_data

In [3]:
tag = input("\n** 원하는 태그를 입력하세요: \n>>> ")
no_of_scroll = input("\n** 스크롤 할 페이지 수를 입력하세요.\n 한 페이지마다 약 35~40개의 데이터가 획득됩니다: \n>>> ")
start_page = input("\n** 스크롤을 시작할 페이지 번호를 입력하세요.\n 맨 처음 데이터부터 가져오고 싶다면 1을 입력하세요: \n>>> ")


** 원하는 태그를 입력하세요: 
>>> 경복궁

** 스크롤 할 페이지 수를 입력하세요.
 한 페이지마다 약 40~45개의 데이터가 획득됩니다: 
>>> 10

** 스크롤을 시작할 페이지 번호를 입력하세요.
 맨 처음 데이터부터 가져오고 싶다면 1을 입력하세요: 
>>> 1


In [4]:
ic = Instagram_crawler()

start = time.time()

ic.crawling_data(tag, int(no_of_scroll), int(start_page))

end = time.time()
seconds = int(end - start)
minutes, seconds = divmod(seconds, 60)
print('  take ', minutes, 'mim', seconds, 'sec')


 http://www.instagram.com/explore/tags/%EA%B2%BD%EB%B3%B5%EA%B6%81 

start to get shortcode
 read the HTML file
 get  51  shortcodes

 read the HTML file
 get  51  shortcodes

 read the HTML file
 get  48  shortcodes

 read the HTML file
 get  48  shortcodes

 read the HTML file
 get  48  shortcodes

 read the HTML file
 get  45  shortcodes

 read the HTML file
 get  45  shortcodes

 read the HTML file
 get  42  shortcodes

 read the HTML file
 get  42  shortcodes

 read the HTML file
 get  42  shortcodes


start to get data
[  1  ]
  https://www.instagram.com/p/BqrEPRaAgzo/
 file saved:  ./dataset/img/경복궁/0.jpg
 Okay

[  2  ]
  https://www.instagram.com/p/BqrEDMvAC3G/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  3  ]
  https://www.instagram.com/p/BqrDpGfgu9t/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  4  ]
  https://www.instagram.com/p/BqrD0MYH3Ua/
 file saved:  ./dataset/img/경복궁/1.jpg
 Okay

[  5  

 file saved:  ./dataset/img/경복궁/50.jpg
 Okay

[  72  ]
  https://www.instagram.com/p/Bqq1FRGB08_/
 file saved:  ./dataset/img/경복궁/51.jpg
 Okay

[  73  ]
  https://www.instagram.com/p/Bqq05sxh9MW/
 file saved:  ./dataset/img/경복궁/52.jpg
 Okay

[  74  ]
  https://www.instagram.com/p/Bqq0kThh7Ty/
 file saved:  ./dataset/img/경복궁/53.jpg
 Okay

[  75  ]
  https://www.instagram.com/p/Bqq0iIcA9PQ/
 file saved:  ./dataset/img/경복궁/54.jpg
 Okay

[  76  ]
  https://www.instagram.com/p/Bqq0Z64gsJk/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  77  ]
  https://www.instagram.com/p/BqqztDsgAIH/
 file saved:  ./dataset/img/경복궁/55.jpg
 Okay

[  78  ]
  https://www.instagram.com/p/BqqzTM8AFu2/
 file saved:  ./dataset/img/경복궁/56.jpg
 Okay

[  79  ]
  https://www.instagram.com/p/BqqykHGHfek/
 file saved:  ./dataset/img/경복궁/57.jpg
 Okay

[  80  ]
  https://www.instagram.com/p/BqqyN7Gg0_S/
 file saved:  ./dataset/img/경복궁/58.jpg
 Okay

[  81  ]
  https://www.instagram.c

 file saved:  ./dataset/img/경복궁/121.jpg
 Okay

[  152  ]
  https://www.instagram.com/p/Bqqa8cVAAtL/
 file saved:  ./dataset/img/경복궁/122.jpg
 Okay

[  153  ]
  https://www.instagram.com/p/BqqaaovnGsV/
 file saved:  ./dataset/img/경복궁/123.jpg
 Okay

[  154  ]
  https://www.instagram.com/p/BqqaIyrgNE2/
 file saved:  ./dataset/img/경복궁/124.jpg
 Okay

[  155  ]
  https://www.instagram.com/p/BqqZ-bcFdPv/
 file saved:  ./dataset/img/경복궁/125.jpg
 Okay

[  156  ]
  https://www.instagram.com/p/BqqZ8XfhmEX/
 file saved:  ./dataset/img/경복궁/126.jpg
 Okay

[  157  ]
  https://www.instagram.com/p/BqkcIg3nJnh/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  158  ]
  https://www.instagram.com/p/BqqZFApH_OU/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  159  ]
  https://www.instagram.com/p/BqqY6w7g-AL/
 file saved:  ./dataset/img/경복궁/127.jpg
 Okay

[  160  ]
  https://www.instagram.com/p/BqqX_eUgAtZ/
 Unexpected error: <class 

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: invalid escape sequence '\/'


 file saved:  ./dataset/img/경복궁/178.jpg
 Okay

[  217  ]
  https://www.instagram.com/p/BqpiOktAT3k/
 file saved:  ./dataset/img/경복궁/179.jpg
 Okay

[  218  ]
  https://www.instagram.com/p/BqpiKMEgUep/
 file saved:  ./dataset/img/경복궁/180.jpg
 Okay

[  219  ]
  https://www.instagram.com/p/BqpiB8sA8M2/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  220  ]
  https://www.instagram.com/p/BqphvkRAs3w/
 file saved:  ./dataset/img/경복궁/181.jpg
 Okay

[  221  ]
  https://www.instagram.com/p/BqphknegoGa/
 file saved:  ./dataset/img/경복궁/182.jpg
 Okay

[  222  ]
  https://www.instagram.com/p/BqphKLpDsiB/
 file saved:  ./dataset/img/경복궁/183.jpg
 Okay

[  223  ]
  https://www.instagram.com/p/BqphJvag7b4/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  224  ]
  https://www.instagram.com/p/BqphIohBfGz/
 file saved:  ./dataset/img/경복궁/184.jpg
 Okay

[  225  ]
  https://www.instagram.com/p/BqphGg1FVyx/
 Unexpected error: <class 

 file saved:  ./dataset/img/경복궁/229.jpg
 Okay

[  291  ]
  https://www.instagram.com/p/BqpUiIHnasT/
 file saved:  ./dataset/img/경복궁/230.jpg
 Okay

[  292  ]
  https://www.instagram.com/p/BqpUCQgFczt/
 file saved:  ./dataset/img/경복궁/231.jpg
 Okay

[  293  ]
  https://www.instagram.com/p/BqpT_u8B_sO/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  294  ]
  https://www.instagram.com/p/BqpTqtBlOpk/
 file saved:  ./dataset/img/경복궁/232.jpg
 Okay

[  295  ]
  https://www.instagram.com/p/BqpTkddhMxf/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  296  ]
  https://www.instagram.com/p/BqpTiliFQHo/
 file saved:  ./dataset/img/경복궁/233.jpg
 Okay

[  297  ]
  https://www.instagram.com/p/BqpTQXlB5Wo/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  298  ]
  https://www.instagram.com/p/BqpTHAPl2zV/
 file saved:  ./dataset/img/경복궁/234.jpg
 Okay

[  299  ]
  https://www.instagram.com/p/Bq

 file saved:  ./dataset/img/경복궁/295.jpg
 Okay

[  369  ]
  https://www.instagram.com/p/BqpDpQygXxd/
 file saved:  ./dataset/img/경복궁/296.jpg
 Okay

[  370  ]
  https://www.instagram.com/p/Bqm01NDBgd-/
 file saved:  ./dataset/img/경복궁/297.jpg
 Okay

[  371  ]
  https://www.instagram.com/p/BqpDlF6HnjJ/
 file saved:  ./dataset/img/경복궁/298.jpg
 Okay

[  372  ]
  https://www.instagram.com/p/BqpCOwzHQVZ/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  373  ]
  https://www.instagram.com/p/BqpCBApnyOq/
 file saved:  ./dataset/img/경복궁/299.jpg
 Okay

[  374  ]
  https://www.instagram.com/p/BqpB1qXDO_t/
 file saved:  ./dataset/img/경복궁/300.jpg
 Okay

[  375  ]
  https://www.instagram.com/p/BqpBr8uBogk/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  376  ]
  https://www.instagram.com/p/BqpBd2uj0Yz/
 file saved:  ./dataset/img/경복궁/301.jpg
 Okay

[  377  ]
  https://www.instagram.com/p/BqpBbaSDm1q/
 file saved:  ./dataset/im

 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  447  ]
  https://www.instagram.com/p/BqoyJIBg-vG/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  448  ]
  https://www.instagram.com/p/BqoyDJkgfeD/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  449  ]
  https://www.instagram.com/p/BqoyCIllVRh/
 file saved:  ./dataset/img/경복궁/362.jpg
 Okay

[  450  ]
  https://www.instagram.com/p/Bqox_oOBypB/
 file saved:  ./dataset/img/경복궁/363.jpg
 Okay

[  451  ]
  https://www.instagram.com/p/Bqox-4EA4uR/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  452  ]
  https://www.instagram.com/p/Bqox6zDgo6I/
 Unexpected error: <class 'ValueError'> 'text' is not in list
 Line  155
 Not okay

[  453  ]
  https://www.instagram.com/p/BqoxwGvha9A/
 file saved:  ./dataset/img/경복궁/364.jpg
 Okay

[  454  ]
  https://www.instagram.com/p/Bqoxot-Ak3r/
 file saved:  .